### BentoML Example
# Titanic Survival Prediction with XGBoost

This is a BentoML Demo Project demonstrating how to package and serve XGBoost model for production using BentoML.

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment.

Let's get started!
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=xgboost&ea=xgboost-tiantic-survival-prediction&dt=xgboost-tiantic-survival-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install bentoml
!pip install xgboost numpy pandas

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import bentoml

# Prepare Dataset
download dataset from https://www.kaggle.com/c/titanic/data

In [3]:
!mkdir data
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv -o ./data/train.csv
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv -o ./data/test.csv

mkdir: data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60302  100 60302    0     0   217k      0 --:--:-- --:--:-- --:--:--  216k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28210  100 28210    0     0   133k      0 --:--:-- --:--:-- --:--:--  133k


In [4]:
train = pd.read_csv("./data/train.csv")
test  = pd.read_csv("./data/test.csv")
X_y_train = xgb.DMatrix(data=train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']], label= train['Survived'])
X_test    = xgb.DMatrix(data=test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])

In [5]:
train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch', 'Survived']].head()

,Pclass,Age,Fare,SibSp,Parch,Survived
0,3,22.0,7.2500,1,0,0
1,1,38.0,71.2833,1,0,1
2,3,26.0,7.9250,0,0,1
3,1,35.0,53.1000,1,0,1
4,3,35.0,8.0500,0,0,0


# Model Training

In [6]:
params = {
          'base_score': np.mean(train['Survived']),
          'eta':  0.1,
          'max_depth': 3,
          'gamma' :3,
          'objective'   :'reg:linear',
          'eval_metric' :'mae'
         }
model = xgb.train(params=params, 
                  dtrain=X_y_train, 
                  num_boost_round=3)

In [7]:
y_test =  model.predict(X_test)
test['pred'] = y_test
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

,Pclass,Age,Fare,SibSp,Parch,pred
10,3,NaN,7.8958,0,0,0.341580
11,1,46.0,26.0000,0,0,0.413966


## Create BentoService for model serving

In [8]:
%%writefile xgboost_titanic_bento_service.py

import xgboost as xgb

import bentoml
from bentoml.artifact import XgboostModelArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([XgboostModelArtifact('model')])
@bentoml.env(pip_dependencies=['xgboost'])
class TitanicSurvivalPredictionService(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler)
    def predict(self, df):
        data = xgb.DMatrix(data=df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])
        return self.artifacts.model.predict(data)

Overwriting xgboost_titanic_bento_service.py


# Save BentoML service archive

In [9]:
# 1) import the custom BentoService defined above
from xgboost_titanic_bento_service import TitanicSurvivalPredictionService

# 2) `pack` it with required artifacts
bento_service = TitanicSurvivalPredictionService()
bento_service.pack('model', model)

# 3) save your BentoSerivce
saved_path = bento_service.save()

[2019-12-11 17:47:09,677] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.3. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
True
[2019-12-11 17:47:09,679] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.3. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/be

no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.5.3+22.g10ffa8c
creating BentoML-0.5.3+22.g10ffa8c/BentoML.egg-info
creating BentoML-0.5.3+22.g10ffa8c/bentoml
creating BentoML-0.5.3+22.g10ffa8c/bentoml/artifact
creating BentoML-0.5.3+22.g10ffa8c/bentoml/bundler
creating BentoML-0.5.3+22.g10ffa8c/bentoml/cli
creating BentoML-0.5.3+22.g10ffa8c/bentoml/clipper
creating BentoML-0.5.3+22.g10ffa8c/bentoml/configuration
creating BentoML-0.5.3+22.g10ffa8c/bentoml/deployment
creating BentoML-0.5.3+22.g10ffa8c/bentoml/deployment/aws_lambda
creating BentoML-0.5.3+22.g10ffa8c/bentoml/deployment/sagemaker
creating BentoML-0.5.3+22.g10ffa8c/bentoml/deployment/serverless
creating BentoML-0.5.3+22.g10ffa8c/bentoml/handlers
creating BentoML-0.5.3+22.g10ffa8c/bentoml/migrations
creating BentoML-0.5.3+22.g10ffa8c/bentoml/migrations/versions
creating BentoML-0.5.3+22.g10ffa8c/bentoml/proto
creating BentoML-0.5.3+22.g10ffa8c/bentoml/repository
creating BentoML-0.5.3+22.g10ffa8c/bentoml/server
creating BentoML-0.5.3+22.g10ffa8c/bentoml

copying bentoml/server/__init__.py -> BentoML-0.5.3+22.g10ffa8c/bentoml/server
copying bentoml/server/bento_api_server.py -> BentoML-0.5.3+22.g10ffa8c/bentoml/server
copying bentoml/server/bento_sagemaker_server.py -> BentoML-0.5.3+22.g10ffa8c/bentoml/server
copying bentoml/server/gunicorn_config.py -> BentoML-0.5.3+22.g10ffa8c/bentoml/server
copying bentoml/server/gunicorn_server.py -> BentoML-0.5.3+22.g10ffa8c/bentoml/server
copying bentoml/server/utils.py -> BentoML-0.5.3+22.g10ffa8c/bentoml/server
copying bentoml/server/static/swagger-ui-bundle.js -> BentoML-0.5.3+22.g10ffa8c/bentoml/server/static
copying bentoml/server/static/swagger-ui.css -> BentoML-0.5.3+22.g10ffa8c/bentoml/server/static
copying bentoml/utils/__init__.py -> BentoML-0.5.3+22.g10ffa8c/bentoml/utils
copying bentoml/utils/cloudpickle.py -> BentoML-0.5.3+22.g10ffa8c/bentoml/utils
copying bentoml/utils/hybirdmethod.py -> BentoML-0.5.3+22.g10ffa8c/bentoml/utils
copying bentoml/utils/log.py -> BentoML-0.5.3+22.g10ffa8c

## Load saved BentoService for serving


In [10]:
import bentoml

bento_model = bentoml.load(saved_path)

result = bento_model.predict(test)
test['pred'] = result
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

[2019-12-11 17:48:15,614] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.3. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2019-12-11 17:48:15,629] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.3,  but loading from BentoML version 0.5.3+22.g10ffa8c
[2019-12-11 17:48:15,632] WARNING - Module `xgboost_titanic_bento_service` already loaded, using existing imported module.
[2019-12-11 17:48:15,638] WARNING - BentoML local changes detected - Local BentoML repository including 

,Pclass,Age,Fare,SibSp,Parch,pred
10,3,NaN,7.8958,0,0,0.341580
11,1,46.0,26.0000,0,0,0.413966


## Model Serving via REST API

In your termnial, run the following command to start the REST API server:

In [14]:
!bentoml serve {saved_path}

[14:20:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
 * Serving Flask app "TitanicSurvivalPredictionService" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Sep/2019 14:20:14] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2019 14:20:16] "POST /predict HTTP/1.1" 200 -
^C


Copy following command to make a curl request to Rest API server

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' \
localhost:5000/predict
```

## Deploy it as API endpoint on AWS Lambda

In order to run this as AWS Lambda function, make sure to configure your AWS credentials via either aws configure command or setting the environment variables below:
```
%env AWS_ACCESS_KEY_ID=
%env AWS_SECRET_ACCESS_KEY=
%env AWS_DEFAULT_REGION=
```
Make sure you have AWS SAM CLI installed:

In [ ]:
%env AWS_ACCESS_KEY_ID=
%env AWS_SECRET_ACCESS_KEY=
%env AWS_DEFAULT_REGION=

Make sure you have AWS SAM CLI installed:

In [ ]:
!pip install -U aws-sam-cli==0.33.1

In [16]:
!bentoml deployment create titanic-prediction \
    --bento=TitanicSurvivalPredictionService:{bento_service.version} \
    --platform=aws-lambda \

[2019-12-12 15:38:26,831] INFO - Building lambda project
[2019-12-12 15:41:29,703] INFO - Packaging AWS Lambda project at /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-_2fjzrcs ...
[2019-12-12 15:41:54,482] INFO - Deploying lambda project
[2019-12-12 15:43:44,620] INFO - ApplyDeployment (titanic-prediction, namespace bobo) succeeded
Successfully created deployment titanic-prediction
{
  "namespace": "bobo",
  "name": "titanic-prediction",
  "spec": {
    "bentoName": "TitanicSurvivalPredictionService",
    "bentoVersion": "20191211174709_CCB40F",
    "operator": "AWS_LAMBDA",
    "awsLambdaOperatorConfig": {
      "region": "us-west-2",
      "memorySize": 1024,
      "timeout": 3
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "endpoints": [
        "https://u7nerrir6a.execute-api.us-west-2.amazonaws.com/Prod/predict"
      ],
      "s3_bucket": "btml-bobo-titanic-prediction-c3da9d"
    },
    "timestamp": "2019-12-12T23:43:44.799944Z"
 

In [13]:
!bentoml deployment describe titanic-prediction

{
  "namespace": "bobo",
  "name": "titanic-prediction",
  "spec": {
    "bentoName": "TitanicSurvivalPredictionService",
    "bentoVersion": "20191211174709_CCB40F",
    "operator": "AWS_LAMBDA",
    "awsLambdaOperatorConfig": {
      "region": "us-west-2",
      "memorySize": 1024,
      "timeout": 3
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "endpoints": [
        "https://eg3tvb519i.execute-api.us-west-2.amazonaws.com/Prod/predict"
      ],
      "s3_bucket": "btml-bobo-titanic-prediction-89a148"
    },
    "timestamp": "2019-12-12T01:54:02.842272Z"
  },
  "createdAt": "2019-12-12T01:49:36.076229Z",
  "lastUpdatedAt": "2019-12-12T01:49:36.076270Z"
}


**To send request to your AWS Lambda deployment, grab the endpoint URL from the json output above:**

In [18]:
!curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' \
https://u7nerrir6a.execute-api.us-west-2.amazonaws.com/Prod/predict














[0.469721257686615]

In [15]:
!bentoml deployment delete titanic-prediction

Successfully deleted deployment "titanic-prediction"
